In [ ]:
"""
Created on Mon Apr 08 16:22 2024

This script is to make a timeseries of runs that have crossed the viability limit

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br.nc')

In [ ]:
ano_choice = xr.open_dataset(inputpath_weights + 'NEMOorISMIP_choice_and_weights.nc')['ano_choice']

In [ ]:
file_TS_scen_list = []
for scenario in ['ssp126','ssp245','ssp585']:
    print(scenario)
    
    file_TS_model_list = []
    for mod in weight_file.model.values:
        print(mod)

        if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
            to2300 = False
        elif mod in ['GISS-E2-1-H']:
            to2300 = True
        elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
            to2300 = True
        elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
            to2300 = False
        elif mod == 'UKESM1-0-LL':
            to2300 = True     
        elif mod == 'CESM2':
            to2300 = False        

        if scenario == 'historical':
            yystart = 1850 #1980 #1850
            yyend = 2014
        elif scenario == 'ssp245':
            yystart = 2015
            yyend = 2100  
        else:
            if to2300:
                yystart = 2015
                yyend = 2300
            else:
                yystart = 2015
                yyend = 2100 

        inputpath_profiles = home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

        T_Clim = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_'+mod+'_clim.nc')
        S_Clim = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_'+mod+'_clim.nc')
        TS_Clim = xr.merge([T_Clim.rename({'thetao':'theta_ocean'}),S_Clim.rename({'so':'salinity_ocean'})])
        
        if scenario == 'ssp126' and mod == 'GFDL-CM4':
            print('beuh')
        else:
        
            file_TS_list = []
            for tt in tqdm(range(yystart,yyend+1)):
                file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
                file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
                file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
                file_TS_list.append(file_TS_orig)
            file_TS_0 = xr.concat(file_TS_list, dim='time') 
            file_TS_model_list.append(file_TS_0.assign_coords({'model': mod}))
        file_TS1 = xr.concat(file_TS_model_list, dim='model')
    file_TS_scen_list.append(file_TS1.assign_coords({'scenario': scenario}))
file_TS2 = xr.concat(file_TS_scen_list, dim='scenario')

In [ ]:
file_TS2.to_netcdf(home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/T_S_prof_allfuturescen_allmod.nc')

In [ ]:
file_TS2 = xr.open_dataset(home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/T_S_prof_allfuturescen_allmod.nc')

In [ ]:
Tmax = file_TS2['theta_ocean'].max('z')

PREPARE THE DATA

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL']

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101)).weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])

In [ ]:
unviable_times = weighted_quantiles > 0
unviable_times_2300 = weighted_quantiles_2300 > 0


In [ ]:
unviable_yy = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101)) > 0
unviable_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=range(1850,2300)).sel(model=model_2300) > 0

In [ ]:
viability_diff = unviable_times.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit_max = viability_diff.time.where(viability_diff == 0).max('time') + 1
limit_max = limit_max.where(limit_max<2100)
limit_max = limit_max.where(~(np.isnan(limit_max) & unviable_times.sel(time=1850)), 1850)

limit_max_full = limit_max.where(np.isfinite(limit_max), 2301)

count_unviable_isf = (unviable_times.time >= limit_max_full).sum('Nisf')

In [ ]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2300)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=1850)), 1850)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')


In [ ]:
unviable_Tmax = Tmax.where(unviable_yy)

In [ ]:
Tmax.where(unviable_yy).sel(Nisf=66,scenario='ssp585')

In [ ]:
f.savefig(plot_path + 'relationship_with_Tocean.png',dpi=300)

In [ ]:
f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

anomaly_mb = file_viability_info['MASS_BALANCE'].sel(time=range(2015,2101)) - file_viability_info['MASS_BALANCE'].sel(time=range(2015,2101)).mean('time')

i = 0

for kisf in tqdm(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):

    ax[i] = f.add_subplot(8,8,i+1)
        
    for scen in ['ssp126','ssp245','ssp585']:

        if scen == 'ssp126':
            ccol = 'gold'
        elif scen == 'ssp245':
            ccol = 'darkorange'
        elif scen == 'ssp585':
            ccol = 'red'
    
        
        for mmod in Tmax.model:
            
            for mparam in file_viability_info['MASS_BALANCE'].param:
                
                for mm in file_viability_info['MASS_BALANCE'].m:
                    
                    ax[i].scatter(Tmax.sel(Nisf=kisf,scenario=scen,time=range(2015,2101),model=mmod),anomaly_mb.sel(model=mmod, Nisf=kisf, scenario=scen, param=mparam, m=mm), s=10, color=ccol, alpha=0.075)
            
    i = i+1
    
    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()

In [ ]:
file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])

In [ ]:
file_viability_info['MASS_BALANCE'].sel(time=range(1850,2101), Nisf=kisf, scenario=scen, param=mparam, m=mm)